# Facial Emotional Recognition Using Shallow CNN Architecture

### Sachin Saj

- This Notebook, deals with facial emotional recognition.
- The dataset used is JAFFE facial emotional dataset
- This dataset consists of 7 facial emotions such as anger, disgust, happy, sad, fear, joy and neutral
- The classification accuracy achieved by the shallow CNN, designed by me was able to achieve same state of the art accuracy of 90.3%, with lesser number of convolution layers (lesser number of learnable parameters) used.
- The result was published in " Symposium on Machine Learning and Metaheuristics Algorithms, and Applications (SoMMA'19) "

In [13]:
from __future__ import print_function
import numpy as np
import cv2
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras import callbacks
from keras.callbacks import CSVLogger
import keras
from keras.models import load_model

In [2]:
batch_size = 20 # Batch Size Used
num_classes = 7 # Seven Emotions Were Considered
epochs = 1000 # Number of Epochs ran

In [3]:
data_augmentation = True

x_train = np.array(np.load("data/jaffe_trainData.npy"))
y_train = np.array(list(map(int, np.load("data/jaffe_trainLabels.npy")))) -1

x_test = np.array(np.load("data/jaffe_testData.npy"))
y_test = np.array(list(map(int, np.load("data/jaffe_testLabels.npy"))))-1

In [4]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [5]:
x_train.shape

(151, 100, 100, 1)

In [6]:
x_test.shape

(62, 100, 100, 1)

In [7]:
input_shape=x_train.shape[1:]
input_shape

(100, 100, 1)

In [10]:
model = Sequential()
model.add(Conv2D(128, (3, 3), padding='same',
input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))	
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 100, 100, 128)     1280      
_________________________________________________________________
activation_6 (Activation)    (None, 100, 100, 128)     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 98, 98, 64)        73792     
_________________________________________________________________
activation_7 (Activation)    (None, 98, 98, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 49, 49, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 49, 49, 64)        36928     
__________

In [11]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [13]:
if not data_augmentation:
	print('Not using data augmentation.')
	model.fit(x_train, y_train,
		batch_size=batch_size,
		epochs=epochs,
		validation_data=(x_test, y_test),
		shuffle=True)
else:
	print('Using real-time data augmentation.')
	# This will do preprocessing and realtime data augmentation:
	datagen = ImageDataGenerator(
		featurewise_center=False, # set input mean to 0 over the dataset
		samplewise_center=False, # set each sample mean to 0
		featurewise_std_normalization=False, # divide inputs by std of the dataset
		samplewise_std_normalization=False, # divide each input by its std
		zca_whitening=False, # apply ZCA whitening
		rotation_range=0, # randomly rotate images in the range (degrees, 0 to 180)
		width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
		height_shift_range=0.1, # randomly shift images vertically (fraction of total height)
		horizontal_flip=True, # randomly flip images
		vertical_flip=False) # randomly flip images
	datagen.fit(x_train)
	

Using real-time data augmentation.


In [15]:
checkpointer = callbacks.ModelCheckpoint(filepath="Results/N_128/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger=CSVLogger('Results/N_128/trainanalysis1.csv',separator=',',append=False)

In [16]:
model.fit_generator(datagen.flow(x_train, y_train,
		batch_size=batch_size),
		steps_per_epoch=x_train.shape[0] // batch_size,
		epochs=epochs,
		validation_data=(x_test, y_test),callbacks=[checkpointer,csv_logger])

Epoch 1/1000
7/7 [==============================] - 49s 7s/step - loss: 2.0388 - acc: 0.0857 - val_loss: 1.9500 - val_acc: 0.1452

Epoch 00001: val_acc improved from -inf to 0.14516, saving model to Results/N_128/checkpoint-01.hdf5
Epoch 2/1000
7/7 [==============================] - 44s 6s/step - loss: 1.9863 - acc: 0.1009 - val_loss: 1.9449 - val_acc: 0.1452

Epoch 00002: val_acc did not improve from 0.14516
Epoch 3/1000
7/7 [==============================] - 44s 6s/step - loss: 1.9453 - acc: 0.1276 - val_loss: 1.9457 - val_acc: 0.1452

Epoch 00003: val_acc did not improve from 0.14516
Epoch 4/1000
7/7 [==============================] - 43s 6s/step - loss: 1.9497 - acc: 0.1564 - val_loss: 1.9464 - val_acc: 0.1452

Epoch 00004: val_acc did not improve from 0.14516
Epoch 5/1000
7/7 [==============================] - 43s 6s/step - loss: 1.9520 - acc: 0.1420 - val_loss: 1.9450 - val_acc: 0.1452

Epoch 00005: val_acc improved from 0.14516 to 0.14516, saving model to Results/N_128/checkpoin

In [27]:
model=load_model ('Results/N_128/checkpoint-424.hdf5')

In [28]:
score = model.evaluate(x_test, y_test)
print("Test accuracy = ",score[1]*100)

62/62 [==============================] - 6s 90ms/step
Test accuracy =  90.32258083743434
